In [20]:
import os
import time
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

%load_ext autoreload
%autoreload 2

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import load

2026-01-03 10:46:11.547923: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-03 10:46:11.548419: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-03 10:46:11.615687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-03 10:46:13.399890: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [12]:
MAX_EPOCHS = 100

def make_save_dir(dirname, experiment_name):
    start_time = str(int(time.time())) + '-' + str(random.randrange(1000))
    save_dir = os.path.join(dirname, experiment_name, start_time)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    return save_dir

def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
            "{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")

In [2]:
params = {
    "conv_subsample_lengths": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    "conv_filter_length": 16,
    "conv_num_filters_start": 32,
    "conv_init": "he_normal",
    "conv_activation": "relu",
    "conv_dropout": 0.2,
    "conv_num_skip": 2,
    "conv_increase_channels_at": 4,

    "learning_rate": 0.001,
    "batch_size": 32,

    "train": "train.json",
    "dev": "dev.json",

    "generator": True,

    "save_dir": "saved"
}

In [8]:
print("Loading training set...")
train = load.load_dataset(params['train'])
ecgs, labels = train
print(f"len(X) : \n {len(ecgs)}")
print(f"len(y) : \n {len(labels)}")
for i in range(10):
    print(f"ecg_{i} : \n {ecgs[i]}")
    print(f"len(ecg_{i}) : {len(ecgs[i])}")
    print(f"label_{i} :\n {labels[i]}")
    print(f"len(label_{i}) : {len(labels[i])}")

Loading training set...


100%|██████████| 7676/7676 [00:01<00:00, 4216.94it/s]

len(X) : 
 7676
len(y) : 
 7676
ecg_0 : 
 [  72   83   93 ... -136 -133 -131]
len(ecg_0) : 8960
label_0 :
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
len(label_0) : 35
ecg_1 : 
 [-137 -167 -200 ...  -50  -51  -51]
len(ecg_1) : 8960
label_1 :
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
len(label_1) : 35
ecg_2 : 
 [620 780 914 ... 102 115 116]
len(ecg_2) : 8960
label_2 :
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
len(label_2) : 35
ecg_3 : 
 [ 96 116 128 ...  45  52  62]
len(ecg_3) : 8960
label_3 :
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N

In [9]:
print("Loading dev set...")
dev = load.load_dataset(params['dev'])

Loading dev set...


100%|██████████| 852/852 [00:00<00:00, 4903.11it/s]


In [40]:
print("Building preprocessor...")
preproc = load.Preproc(*train)
print("Training size: " + str(len(train[0])) + " examples.")
print("Dev size: " + str(len(dev[0])) + " examples.")

Building preprocessor...
self.mean :  7.4661856  self.std :  236.10312
self.classes :  ['A', 'N', 'O', '~']
self.int_to_class :  {0: 'A', 1: 'N', 2: 'O', 3: '~'}
self.class_to_int :  {'A': 0, 'N': 1, 'O': 2, '~': 3}
Training size: 7676 examples.
Dev size: 852 examples.


In [21]:
from resnet1d import MyDataset, ResNet1D

train_dataset = MyDataset(*train)

In [23]:
data,labels = train

In [27]:
x,y = data[:5], labels[:5]

In [28]:
x

[array([  72,   83,   93, ..., -136, -133, -131],
       shape=(8960,), dtype=int16),
 array([-137, -167, -200, ...,  -50,  -51,  -51],
       shape=(8960,), dtype=int16),
 array([620, 780, 914, ..., 102, 115, 116], shape=(8960,), dtype=int16),
 array([ 96, 116, 128, ...,  45,  52,  62], shape=(8960,), dtype=int16),
 array([702, 837, 986, ..., 150, 147, 143], shape=(8960,), dtype=int16)]

In [33]:
y[2]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [31]:
for el in y : 
    print(len(el))

35
35
35
35
35


In [30]:
preproc.process_x(x)

array([[[ 0.27332893],
        [ 0.31991875],
        [ 0.36227313],
        ...,
        [-0.60764205],
        [-0.5949358 ],
        [-0.5864649 ]],

       [[-0.6118775 ],
        [-0.7389406 ],
        [-0.8787101 ],
        ...,
        [-0.24339443],
        [-0.24762988],
        [-0.24762988]],

       [[ 2.5943487 ],
        [ 3.2720187 ],
        [ 3.8395672 ],
        ...,
        [ 0.40039206],
        [ 0.45545274],
        [ 0.4596882 ]],

       [[ 0.37497944],
        [ 0.4596882 ],
        [ 0.5105134 ],
        ...,
        [ 0.15897211],
        [ 0.18862018],
        [ 0.23097456]],

       [[ 2.9416544 ],
        [ 3.5134387 ],
        [ 4.144519  ],
        ...,
        [ 0.60369307],
        [ 0.5909867 ],
        [ 0.574045  ]]], shape=(5, 8960, 1), dtype=float32)

In [75]:
from load_data import ECGDataset, ECGCollate, SmartBatchSampler

train_dataset = ECGDataset(*train)


MEAN :  7.4661856  STD :  236.10312
self.classes :  ['A', 'N', 'O', '~']
self.class_to_int :  {'A': 0, 'N': 1, 'O': 2, '~': 3}


In [79]:

# Instanciation du Sampler intelligent
batch_sampler = SmartBatchSampler(dataset, 32)

collate_fn = ECGCollate(
    pad_val_x=dataset.pad_value_x_normalized,
    num_classes=dataset.num_classes
)

loader = DataLoader(
    dataset,
    # IMPORTANT : Quand on utilise batch_sampler, on ne met ni batch_size, ni shuffle, ni drop_last
    # dans le DataLoader car le Sampler gère tout ça.
    batch_sampler=batch_sampler, 
    collate_fn=collate_fn,
    num_workers=4
)
    

Tri du dataset par longueur pour minimiser le padding...


In [77]:
dataset[2]

(tensor([2.5943, 3.2720, 3.8396,  ..., 0.4004, 0.4555, 0.4597]),
 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]))

In [50]:
17920/256

70.0

In [81]:
for batch in loader : 
    x,y = batch
    print(x.shape, y.shape)

torch.Size([32, 1, 5632]) torch.Size([32, 22])
torch.Size([32, 1, 4864]) torch.Size([32, 19])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 9472]) torch.Size([32, 37])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 7936]) torch.Size([32, 31])
torch.Size([32, 1, 17920]) torch.Size([32, 70])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 17920]) torch.Size([32, 70])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 8960]) torch.Size([32, 35])
torch.Size([32, 1, 9728]) torch.Size([32, 38])
torch.Size(

In [82]:
x

tensor([[[ 0.6926,  0.8875,  1.1119,  ..., -0.4975, -0.4890, -0.4806]],

        [[ 0.0658,  0.0954,  0.1251,  ..., -0.2942, -0.2773, -0.2646]],

        [[ 0.1505,  0.1886,  0.2225,  ...,  2.4927,  2.5096,  2.5223]],

        ...,

        [[ 0.9383,  1.2263,  1.5143,  ..., -0.4298, -0.4086, -0.3874]],

        [[ 0.1039,  0.2183,  0.2522,  ..., -0.0316, -0.0316, -0.0316]],

        [[-2.9244, -3.6063, -4.3433,  ...,  0.1463,  0.1420,  0.1378]]])

In [83]:
y

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [2, 2, 2,  ..., 2, 2, 2],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [ ]:
import argparse  # Module pour gérer les arguments passés en ligne de commande

# Création du parseur d'arguments
parser = argparse.ArgumentParser()

# Argument positionnel obligatoire : chemin vers un fichier de configuration
parser.add_argument(
    "config_file",
    help="path to config file"
)

# Argument optionnel : nom de l'expérience
# Peut être fourni avec --experiment ou -e
# Valeur par défaut : "default"
parser.add_argument(
    "--experiment",
    "-e",
    help="tag with experiment name",
    default="default"
)

# Analyse des arguments fournis lors de l'exécution du script
args = parser.parse_args()

# Après cette ligne :
# args.config_file contient le chemin du fichier de configuration
# args.experiment contient le nom de l'expérience

In [19]:
import util

save_dir = make_save_dir(params['save_dir'], "test_run")

util.save(preproc, save_dir)

params.update({
    "input_shape": [None, 1],
    "num_categories": len(preproc.classes)
})

print(params)

{'conv_subsample_lengths': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2], 'conv_filter_length': 16, 'conv_num_filters_start': 32, 'conv_init': 'he_normal', 'conv_activation': 'relu', 'conv_dropout': 0.2, 'conv_num_skip': 2, 'conv_increase_channels_at': 4, 'learning_rate': 0.001, 'batch_size': 32, 'train': 'train.json', 'dev': 'dev.json', 'generator': True, 'save_dir': 'saved', 'input_shape': [None, 1], 'num_categories': 4}


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs --port 6006

In [ ]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter


def create_tqdm_bar(iterable, desc):
    return tqdm(enumerate(iterable),total=len(iterable), ncols=150, desc=desc)


def train_model(model, train_loader, val_loader, loss_func, tb_logger, epochs=10, name="default"):
    """
    Train the classifier for a number of epochs.
    """
    loss_cutoff = len(train_loader) // 10
    optimizer = torch.optim.Adam(model.parameters(), params["learning_rate"])

    # The scheduler is used to change the learning rate every few "n" steps.
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(epochs * len(train_loader) / 5), gamma=hparams.get('gamma', 0.8))

    for epoch in range(epochs):

        # Training stage, where we want to update the parameters.
        model.train()  # Set the model to training mode

        training_loss = []
        validation_loss = []

        # Create a progress bar for the training loop.
        training_loop = create_tqdm_bar(train_loader, desc=f'Training Epoch [{epoch + 1}/{epochs}]')
        for train_iteration, batch in training_loop:
            optimizer.zero_grad() # Reset the gradients - VERY important! Otherwise they accumulate.
            images, labels = batch # Get the images and labels from the batch, in the fashion we defined in the dataset and dataloader.
            images, labels = images.to(device), labels.to(device) # Send the data to the device (GPU or CPU) - it has to be the same device as the model.

            # Flatten the images to a vector. This is done because the classifier expects a vector as input.
            # Could also be done by reshaping the images in the dataset.
            images = images.view(images.shape[0], -1)

            pred = model(images) # Stage 1: Forward().
            loss = loss_func(pred, labels) # Compute the loss over the predictions and the ground truth.
            loss.backward()  # Stage 2: Backward().
            optimizer.step() # Stage 3: Update the parameters.
            scheduler.step() # Update the learning rate.


            training_loss.append(loss.item())
            training_loss = training_loss[-loss_cutoff:]

            # Update the progress bar.
            training_loop.set_postfix(curr_train_loss = "{:.8f}".format(np.mean(training_loss)),
                                      lr = "{:.8f}".format(optimizer.param_groups[0]['lr'])
            )

            # Update the tensorboard logger.
            tb_logger.add_scalar(f'classifier_{name}/train_loss', loss.item(), epoch * len(train_loader) + train_iteration)

        # Validation stage, where we don't want to update the parameters. Pay attention to the classifier.eval() line
        # and "with torch.no_grad()" wrapper.
        model.eval()
        val_loop = create_tqdm_bar(val_loader, desc=f'Validation Epoch [{epoch + 1}/{epochs}]')

        with torch.no_grad():
            for val_iteration, batch in val_loop:
                images, labels = batch
                images, labels = images.to(device), labels.to(device)

                images = images.view(images.shape[0], -1)
                pred = model(images)
                loss = loss_func(pred, labels)
                validation_loss.append(loss.item())
                # Update the progress bar.
                val_loop.set_postfix(val_loss = "{:.8f}".format(np.mean(validation_loss)))

                # Update the tensorboard logger.
                tb_logger.add_scalar(f'classifier_{name}/val_loss', loss.item(), epoch * len(val_loader) + val_iteration)


In [ ]:
# Create a tensorboard logger.
# NOTE: In order to see the logs, run the following command in the terminal: tensorboard --logdir=./
# Also, in order to reset the logs, delete the logs folder MANUALLY.

path = "logs"
num_of_runs = len(os.listdir(path)) if os.path.exists(path) else 0
path = os.path.join(path, f'run_{num_of_runs + 1}')

tb_logger = SummaryWriter(path)

# Train the classifier.
train_loader = data_module.train_dataloader()
labled_val_loader = data_module.val_dataloader()

epochs = hparams.get('epochs', 4)
loss_func = nn.CrossEntropyLoss() # The loss function we use for classification.
# make model
device_str = "cuda"
device = torch.device(device_str if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")

kernel_size = 16 # 16 in Hannun et al.
stride = 2
n_block = 16 # 16 in Hannun et al.
downsample_gap = 2 # 2 in Hannun et al.
increasefilter_gap = 4 # 4 in Hannun et al.

model = ResNet1D(
    in_channels=1, 
    base_filters=32, # 32 in Hannun et al.
    kernel_size=kernel_size, 
    stride=stride, 
    groups=1, # like a classical ResNet
    n_block=n_block, 
    n_classes=2, 
    downsample_gap=downsample_gap, 
    increasefilter_gap=increasefilter_gap, 
    use_bn=True,
    use_do=True,
    verbose=True
    )

model.to(device)

summary(model, (X_train.shape[1], X_train.shape[2]), device=device_str)
train_model(model, labled_train_loader, labled_val_loader, loss_func, tb_logger, epochs=epochs, name="Default")

print()
print("Finished training!")
print("How did we do? Let's check the accuracy of the defaut classifier on the training and validation sets:")
print(f"Training Acc: {model.getTestAcc(labled_train_loader)[1] * 100}%")
print(f"Validation Acc: {model.getTestAcc(labled_val_loader)[1] * 100}%")